In [22]:
# things to note:
# setup file creation for installing packages: https://stackoverflow.com/a/4527622/8324498

In [23]:
# import statements
import boto3
import json

In [24]:
# initialize quicksight
qs_client = boto3.client('quicksight')

In [25]:
# initialize the variables
aws_account_id = '733585711144'
region = 'us-east-1'
dashboard_id = 'd280fdec-2adf-45bb-813d-cceece251ec3'
target_dataset_id = 'b40f8cfd-adb1-40c4-b972-b4a0289d7a56'
target_dataset_identifier = 'Strategic'
target_dataset_arn = f'arn:aws:quicksight:{region}:{aws_account_id}:dataset/{target_dataset_id}'
sheet_id = 'd280fdec-2adf-45bb-813d-cceece251ec3_642ba41e-f491-4357-bf4a-a2d4eb9638bc'
visual_id = 'd280fdec-2adf-45bb-813d-cceece251ec3_52d335ed-bbbc-46bd-98d6-000c81a214e6'
replacement_url = 'https://tiny.amazon.com/1dz1oyrsy/aacbs3useaamazawspng'

In [26]:
# target dashboard parameters
target_dashboard_id = 'StrategicDashboard'
target_dashboard_name = 'Strategic'

In [27]:
# describe dashbboard
dashboard_definition = qs_client.describe_dashboard_definition(AwsAccountId=aws_account_id, DashboardId=dashboard_id)

In [28]:
# Optional: download definition
with open('./enterpriseDashboard.json', "w") as outfile:
    json.dump(dashboard_definition, outfile, indent=4)

In [29]:
# cleanup residue
# for key in ['Status', 'Name', 'DashboardId', 'RequestId', 'ResponseMetadata']:
#     dashboard_definition.pop(key)

In [30]:
def update_nested_dict(in_dict, key, value):
    """Replaces the existing value of the key with a new value

    Args:
        in_dict (dict): dictionary to be executed
        key (str): key to search for; example 'DataSetIdentifier' or 'Identifier'...
        value (str): value to replace with; example 'NewValue'

    Returns:
        doesn't return anything but updates the dictionary in place
    """
    for k, v in in_dict.items():
        if key == k:
            in_dict[k] = value
        elif isinstance(v, dict):
            update_nested_dict(v, key, value)
        elif isinstance(v, list):
            for o in v:
                if isinstance(o, dict):
                    update_nested_dict(o, key, value)

In [31]:
# creating object dictionaries
dataset_dict = dashboard_definition['Definition']['DataSetIdentifierDeclarations'][0]
sheets_dict = dashboard_definition['Definition']['Sheets']
for sheet in sheets_dict:
    if sheet['SheetId'] == sheet_id:
        visuals_in_sheet = sheet['Visuals']
        break

# update dataset name and arn in the definition
update_nested_dict(dataset_dict, 'Identifier', target_dataset_identifier)
update_nested_dict(dataset_dict, 'DataSetArn', target_dataset_arn)
update_nested_dict(dashboard_definition, 'DataSetIdentifier', target_dataset_identifier)

In [32]:
# Optional: store the resultant json locally
with open('./modifiedJson.json', 'w') as outfile:
    json.dump(dashboard_definition, outfile, indent=4)

In [33]:
# update visual URL in your JSON
for visual in visuals_in_sheet:
    try:
       if visual['CustomContentVisual']['VisualId'] == visual_id:
        visual['CustomContentVisual']['ChartConfiguration']['ContentUrl'] = replacement_url

    except KeyError:
        continue

In [34]:
# get dashboard permissions
dashboard_permissions = qs_client.describe_dashboard_permissions(AwsAccountId=aws_account_id, DashboardId=dashboard_id)['Permissions']

In [35]:
# create target dashboard
qs_client.create_dashboard(
        AwsAccountId=aws_account_id,
        DashboardId=target_dashboard_id,
        Name=target_dashboard_name,
        Definition=dashboard_definition['Definition'],
        Permissions=dashboard_permissions
    )


{'ResponseMetadata': {'RequestId': 'ed1ac878-3bef-4b12-b1af-91c66d9174c0',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 26 Jan 2023 18:40:39 GMT',
   'content-type': 'application/json',
   'content-length': '302',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ed1ac878-3bef-4b12-b1af-91c66d9174c0'},
  'RetryAttempts': 0},
 'Status': 202,
 'Arn': 'arn:aws:quicksight:us-east-1:733585711144:dashboard/StrategicDashboard',
 'VersionArn': 'arn:aws:quicksight:us-east-1:733585711144:dashboard/StrategicDashboard/version/1',
 'DashboardId': 'StrategicDashboard',
 'CreationStatus': 'CREATION_IN_PROGRESS',
 'RequestId': 'ed1ac878-3bef-4b12-b1af-91c66d9174c0'}

In [36]:
# check dashboard status
qs_client.describe_dashboard(AwsAccountId=aws_account_id, DashboardId=target_dashboard_id)

{'ResponseMetadata': {'RequestId': 'f4c5658f-7c1e-498f-926a-5581d08b4bda',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 26 Jan 2023 18:40:39 GMT',
   'content-type': 'application/json',
   'content-length': '1151',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f4c5658f-7c1e-498f-926a-5581d08b4bda'},
  'RetryAttempts': 0},
 'Status': 200,
 'Dashboard': {'DashboardId': 'StrategicDashboard',
  'Arn': 'arn:aws:quicksight:us-east-1:733585711144:dashboard/StrategicDashboard',
  'Name': 'Strategic',
  'Version': {'CreatedTime': datetime.datetime(2023, 1, 26, 18, 40, 39, 861000, tzinfo=tzlocal()),
   'Errors': [],
   'VersionNumber': 1,
   'Status': 'CREATION_SUCCESSFUL',
   'DataSetArns': ['arn:aws:quicksight:us-east-1:733585711144:dataset/b40f8cfd-adb1-40c4-b972-b4a0289d7a56'],
   'Sheets': [{'SheetId': 'd280fdec-2adf-45bb-813d-cceece251ec3_642ba41e-f491-4357-bf4a-a2d4eb9638bc',
     'Name': 'Sheet 1'}]},
  'CreatedTime': datetime.datetime(2023, 1, 26, 18, 40, 39, 457000